In [7]:
import pandas as pd
import re
import string
import random
from transformers import BertTokenizer, BertForMultipleChoice
from transformers import BertForSequenceClassification
import torch.nn as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from transformers import AdamW 
import json
import torch
import numpy as np
import math
test = pd.read_csv('./sample_submission.csv')
model_path = './bert/'

In [4]:
doc = pd.read_csv('./documents.csv')
doc_list = doc['doc_id'].tolist()
doc_text = doc['doc_text'].tolist()
doc_dict = {doc_list[i]: i for i in range(len(doc_list))}
# print(len(doc_text))

In [10]:
train_q = pd.read_csv('./train_queries.csv')
train_query_data = [t.lower() for t in train_q['query_text']]
train_top_1000 = [t.split() for t in train_q['bm25_top1000']]
train_pos_list = [t.split() for t in train_q['pos_doc_ids']]
train_neg_list = [t[:] for t in train_top_1000] 
train_pos_list = np.array(train_pos_list)
train_top_1000 = np.array(train_top_1000)
train_neg_list = np.array(train_neg_list) 

/home/nlp/.local/lib/python3.7/site-packages/ipykernel_launcher.py:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  


In [5]:
# ## 裡面現在存的是pos doc idx
# for pos_docs in range(train_pos_list.shape[0]):
#     for idx in range(len(train_pos_list[pos_docs])):
#         train_pos_list[pos_docs][idx] = doc_dict[train_pos_list[pos_docs][idx]]
        
# for pos_docs in range(train_top_1000.shape[0]):
#     for idx in range(len(train_top_1000[pos_docs])):
#         train_top_1000[pos_docs][idx] = doc_dict[train_top_1000[pos_docs][idx]]
        
# for pos_docs in range(train_neg_list.shape[0]):
#     for idx in range(len(train_neg_list[pos_docs])):
#         train_neg_list[pos_docs][idx] = doc_dict[train_neg_list[pos_docs][idx]]


In [13]:
print(relevant[0])
print(document[0])

['FBIS3-20548', 'FBIS3-22539', 'FBIS3-22560', 'FBIS3-22589', 'FBIS3-26593', 'FBIS3-41672', 'FBIS3-41724', 'FBIS3-60403', 'FBIS3-60404', 'FBIS3-60405', 'FBIS3-60419', 'FBIS3-60450', 'FBIS3-60503', 'FBIS3-60510', 'FBIS3-60561', 'FBIS3-60562', 'FBIS3-61373', 'FBIS3-9904', 'FBIS4-22716', 'FBIS4-24438', 'FBIS4-2880', 'FBIS4-30637', 'FBIS4-33063', 'FBIS4-33435', 'FBIS4-38133', 'FBIS4-4241', 'FBIS4-45613', 'FBIS4-45844', 'FBIS4-5325', 'FBIS4-57220', 'FBIS4-67701', 'FBIS4-67707', 'FBIS4-67720', 'FR940126-2-00106', 'FR940207-2-00089', 'FR940425-2-00078', 'FR940425-2-00079', 'FR940425-2-00080', 'FR940603-2-00060', 'FR940620-2-00117', 'FT921-14537', 'FT921-16061', 'FT921-7765', 'FT924-10652', 'FT931-11085', 'FT931-7060', 'FT932-4485', 'FT934-1206', 'FT941-12410', 'FT944-2489', 'FT944-2592', 'LA010589-0059', 'LA012490-0111', 'LA012589-0035', 'LA031190-0116', 'LA031489-0032', 'LA043090-0036', 'LA061289-0041', 'LA072890-0066', 'LA080690-0099', 'LA082490-0065', 'LA090990-0031', 'LA100189-0047', 'LA12

In [16]:
N = 20
document, relevant = train_top_1000, train_pos_list
# for i in range(N):
#     document.append(input())
#     relevant.append(input())

MAP = 0.0
for index in range(N):
    relevant_dict = {} # 將 relevant 的資料存成 dictionary 以防有重複的 word
    precision = relevant[index]
    for j in precision:
        relevant_dict[j] = 0

    score = 0.0
    ans = 0
    doc = document[index]

    for i in range(len(doc)):
        if doc[i] in relevant_dict: # 若文章中的字在 relevant doc 中有出現，則計算 score
            ans += 1
            score += ans / (i+1)  # 總共出現相關文章的次數 / 現在的文章數量

    MAP += score / len(precision) # 除以所有 relevant document 的數量

MAP /= N # 除以 document 的數量
MAP = round(MAP, 4)
print(MAP)

0.2599


In [20]:
def MAP():
    top_1000 = [t.split() for t in train_q['bm25_top1000']]
    pos_ans = [t.split() for t in train_q['pos_doc_ids']]
    num = 0
    a = 0 
    num_query = 120 ## 25 拿來test 
    all_score = 0.0
    for i in range(int(num_query)):
        score = 0.0
        find_list = []## all list
        pred_list = [] ## all pred score
        pos_list = [] ##all score changed pos 
        relevant_list = {} ## all relevant
#         all_find = input() ##所有query結果
        all_find_split = top_1000[i] ## list all find
        for j in top_1000[i]:
            find_list.append(j) ## 建list
            pred_list.append(0)
#         relevant = input()
        all_relevant_split = pos_ans[i] ## list all find
        print(len(all_relevant_split))
        num+=len(all_relevant_split)
        if len(all_relevant_split)>30:
            a += 1
        for j in pos_ans[i]:
            relevant_list[j] = 0 ## 建dict 比較好查找
        now_pos = 0
        correct = 0 ##對幾個
        count = 0
        for j in range(len(find_list)):
            count += 1
            if(find_list[j] in relevant_list):
                correct += 1
                for k in range(count-now_pos):
                    pred_list[now_pos+k] = correct/count
                now_pos = j+1
                pos_list.append(j)
        for j in range(len(pos_list)):
            score += pred_list[pos_list[j]]
        if len(pos_list)!=0:
            score /= len(all_relevant_split) 
        all_score += score
    all_score/=int(num_query)
    print(num)
    print(a)
    return (round(all_score,4))

In [21]:
test_a = [t.split() for t in train_q['bm25_top1000']]
MAP() ##要傳25x1000

65
35
210
4
3
13
11
113
170
203
34
61
161
24
46
11
8
35
60
254
9
66
12
69
78
20
73
68
48
246
361
17
270
51
28
151
16
35
61
13
17
49
33
80
102
98
22
146
51
19
85
51
194
134
178
105
71
17
213
27
145
102
125
300
118
22
65
69
136
75
116
16
33
83
171
162
118
6
28
13
347
117
173
219
54
17
94
102
17
46
67
293
5
84
16
72
14
27
30
12
68
21
9
18
27
12
19
115
71
47
11
19
14
32
43
49
39
57
37
34
9356
79


0.317